In [ ]:
%matplotlib inline

# imports 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import flowkit as fk 
from openTSNE import TSNE

sample = fk.Sample('data/flow18_annotated.fcs', sample_id='flow18', channel_labels=('Parameter_1', 'Parameter_10', 'Parameter_11', 'Parameter_12', 'Parameter_13', 'Parameter_14', 'Parameter_15', 'Parameter_16', 'Parameter_17', 'Parameter_18', 'Parameter_19', 'Parameter_2', 'Parameter_20', 'Parameter_21', 'Parameter_22', 'Parameter_23', 'Parameter_24', 'Parameter_3', 'Parameter_4', 'Parameter_5', 'Parameter_6', 'Parameter_7', 'Parameter_8', 'Parameter_9', 'SampleID', 'class'))
df_events = sample.as_dataframe(source="raw")

# only use selected columns 
selected_columns = [
    'Parameter_10', 'Parameter_11', 'Parameter_12', 
    'Parameter_13', 'Parameter_15', 'Parameter_18', 'Parameter_20', 
    'Parameter_21', 'Parameter_23', 'Parameter_8', 'Parameter_9', 'class'
]

df_filtered = df_events[selected_columns]

fraction_used = 1.0 

# sample from dataset 
data_sample = df_filtered.sample(frac=fraction_used).reset_index(drop=True)

# get features and labels 
features = data_sample.drop(columns=['class'])  # All columns except 'class'
labels = data_sample['class']  # Only the 'class' column

In [ ]:
# step 1: define affinities 
import openTSNE 
affinities12 = openTSNE.affinity.PerplexityBasedNN(
    features.to_numpy(),
    perplexity=50, 
    n_jobs=-1,
    random_state=12,
    verbose=True
    )

affinities42 = openTSNE.affinity.PerplexityBasedNN(
    features.to_numpy(),
    perplexity=50, 
    n_jobs=-1,
    random_state=42,
    verbose=True
    )

affinities3 = openTSNE.affinity.PerplexityBasedNN(
    features.to_numpy(),
    perplexity=50, 
    n_jobs=-1,
    random_state=3,
    verbose=True
    )

In [ ]:
init12 = openTSNE.initialization.random(n_samples=len(features) ,random_state=12, verbose=True)
init42 = openTSNE.initialization.random(n_samples=len(features) ,random_state=42, verbose=True)
init3 = openTSNE.initialization.random(n_samples=len(features) ,random_state=3, verbose=True)

In [ ]:
# step 3: construct TSNEEmbedding object 
embedding12 = openTSNE.TSNEEmbedding(
    init12, 
    affinities12, 
    random_state=12,
    verbose=True
)

embedding42 = openTSNE.TSNEEmbedding(
    init42, 
    affinities42, 
    random_state=12,
    verbose=True
)

embedding3 = openTSNE.TSNEEmbedding(
    init3, 
    affinities3, 
    random_state=12,
    verbose=True
)

In [ ]:
embedding12 = embedding12.optimize(
    n_iter=250, 
    learning_rate=200, 
    exaggeration=12, 
    momentum=0.5,  
    verbose=True
)

embedding42 = embedding42.optimize(
    n_iter=250, 
    learning_rate=200, 
    exaggeration=12, 
    momentum=0.5,  
    verbose=True
)

embedding3 = embedding3.optimize(
    n_iter=250, 
    learning_rate=200, 
    exaggeration=12, 
    momentum=0.5,  
    verbose=True
)

In [ ]:
embedding12_final = embedding12.optimize(
    n_iter=750, 
    learning_rate=200, 
    momentum=0.8, 
    verbose=True
)

embedding42_final = embedding42.optimize(
    n_iter=750, 
    learning_rate=200, 
    momentum=0.8, 
    verbose=True
)

embedding3_final = embedding3.optimize(
    n_iter=750, 
    learning_rate=200, 
    momentum=0.8, 
    verbose=True
)

In [ ]:
# List of embeddings
all_embeddings = [embedding12, embedding12_final, embedding42, embedding42_final, embedding3, embedding3_final]

# Create a 2x2 grid
fig, axes = plt.subplots(2, 3, figsize=(10, 10))

# Title for the entire figure
fig.suptitle("Impact of different seed values on embedding", fontsize=16, y=0.92)

# Subtitles for each subplot
subtitles = ["seed 12 EE", "seed 12", "seed 42 EE", "seed 42", "seed 3 EE", "seed 3"]

# Plot each embedding in the corresponding subplot
for ax, embedding, subtitle in zip(axes.flat, all_embeddings, subtitles):
    scatter = ax.scatter(embedding[:, 0], embedding[:, 1],
                          c=labels, cmap=plt.colormaps.get_cmap('Paired'), s=10, alpha=0.8)
    ax.set_xticks([])  # Hide x ticks
    ax.set_yticks([])  # Hide y ticks
    ax.set_title(subtitle, fontsize=12)  # Subtitle for each subplot

#plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to include the main title
plt.show()